In [1]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import os
from datetime import datetime
import darts as dt
import seaborn as sns

In [2]:
ticker = yf.Ticker(ticker='AAPL').history('1y')

In [3]:
filepath = "/Users/mudit.000/OneDrive - University of Essex/Documents/GitHub/Personal-Research-Learning/Personal-Research-Learning/NSE-Stock-Historical-Data/v1"

In [4]:
dp = pl.read_csv(filepath + "/3IINFOLTD.NS.csv")

In [5]:
stocklist = []
stocknames = []

In [6]:
for files in os.listdir(filepath):
    stocknames.append(files)
    stocklist.append(pl.read_csv(filepath + "/" + files))

In [7]:
stocknames = pl.Series(stocknames)

In [8]:
stocknames = stocknames.str.replace(".NS.csv", "")

In [9]:
stocknames

""
str
"""20MICRONS"""
"""21STCENMGM"""
"""360ONE"""
"""3IINFOLTD"""
"""3MINDIA"""
…
"""ZOTA"""
"""ZUARI"""
"""ZUARIIND"""


In [10]:
df = dp.to_pandas()

In [11]:
dp = dp.with_columns(pl.col("Date").str.strptime(dtype=pl.Datetime).alias("Date"))

In [12]:
df.dtypes

Date             object
Open            float64
High            float64
Low             float64
Close           float64
Volume            int64
Dividends       float64
Stock Splits    float64
dtype: object

In [13]:
df['Date'] = df['Date'].str.slice(start=0, stop=11)

In [14]:
temp = []
for x in stocklist:
    temp.append(x.with_columns(pl.col('Date').str.slice(0,10).alias('Date')))

stocklist = temp

In [15]:
temp = pl.DataFrame()
i = 0
for x in stocklist:
    try:
        temp = temp.vstack(x)
        i += 1
    except:
        print(i)
        break

25


In [16]:
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2005-04-22,520.664895,520.664895,423.591779,432.857849,3304177,0.0,0.0
1,2005-04-25,436.387792,439.917724,415.649444,419.002869,1021511,0.0,0.0
2,2005-04-26,422.091553,424.739001,410.795783,412.119507,680155,0.0,0.0
3,2005-04-27,412.119501,449.007275,407.530580,424.033020,1872361,0.0,0.0
4,2005-04-28,423.591803,435.505322,417.679156,421.650330,741207,0.0,0.0
...,...,...,...,...,...,...,...,...
4569,2023-10-26,36.000000,36.599998,34.650002,35.700001,770037,0.0,0.0
4570,2023-10-27,36.099998,36.950001,35.900002,36.599998,573842,0.0,0.0
4571,2023-10-30,36.799999,37.349998,36.250000,36.900002,400942,0.0,0.0
4572,2023-10-31,37.099998,37.450001,36.200001,36.400002,353270,0.0,0.0


In [17]:
logdf = np.log(df["Close"] ) - np.log(df["Open"])

In [18]:
temp = []
for x in stocklist:
    temp.append(pl.Series(np.log(x["Close"] ) - np.log(x["Open"])))
logstock = temp

c:\Users\mudit.000\OneDrive - University of Essex\Documents\GitHub\Personal-Research-Learning\Personal-Research-Learning\.venv\Lib\site-packages\polars\series\series.py:1458: RuntimeWarning: invalid value encountered in log
  lambda out: ufunc(*args, out=out, dtype=dtype_char, **kwargs),
c:\Users\mudit.000\OneDrive - University of Essex\Documents\GitHub\Personal-Research-Learning\Personal-Research-Learning\.venv\Lib\site-packages\polars\series\series.py:1458: RuntimeWarning: divide by zero encountered in log
  lambda out: ufunc(*args, out=out, dtype=dtype_char, **kwargs),


In [19]:
stocklist[6]

Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
str,f64,f64,f64,f64,i64,f64,f64
"""2018-01-19""",104.199997,104.199997,104.199997,104.199997,36400,0.0,0.0
"""2018-01-22""",104.199997,104.199997,104.199997,104.199997,0,0.0,0.0
"""2018-01-23""",104.199997,104.199997,104.199997,104.199997,0,0.0,0.0
"""2018-01-24""",104.199997,104.199997,104.199997,104.199997,0,0.0,0.0
"""2018-01-25""",104.199997,104.199997,104.199997,104.199997,0,0.0,0.0
…,…,…,…,…,…,…,…
"""2023-10-25""",64.599998,64.599998,64.599998,64.599998,0,0.0,0.0
"""2023-10-26""",64.599998,64.599998,64.599998,64.599998,0,0.0,0.0
"""2023-10-27""",64.599998,64.599998,64.599998,64.599998,0,0.0,0.0


In [20]:
logstock

[shape: (3_714,)
 Series: 'Close' [f64]
 [
 	-0.86899
 	-0.062873
 	-0.05506
 	-0.070715
 	0.014301
 	…
 	0.0
 	0.0
 	0.0
 	0.0
 	0.0
 ],
 shape: (1_434,)
 Series: 'Close' [f64]
 [
 	0.0
 	0.0
 	0.0
 	0.0
 	0.0
 	…
 	0.01087
 	-0.01087
 	0.002743
 	-0.021859
 	-0.008392
 ],
 shape: (1_022,)
 Series: 'Close' [f64]
 [
 	0.04879
 	0.003755
 	0.0
 	0.0
 	-0.094504
 	…
 	0.000672
 	-0.005327
 	-0.008935
 	0.003052
 	0.007026
 ],
 shape: (4_574,)
 Series: 'Close' [f64]
 [
 	-0.184697
 	-0.040654
 	-0.023909
 	0.028498
 	-0.004594
 	…
 	-0.008368
 	0.013755
 	0.002714
 	-0.019048
 	0.024424
 ],
 shape: (5_300,)
 Series: 'Close' [f64]
 [
 	0.013152
 	-0.00692
 	-0.000509
 	0.091478
 	-0.03767
 	…
 	-0.004684
 	0.022893
 	-0.00963
 	0.003792
 	0.012689
 ],
 shape: (3_838,)
 Series: 'Close' [f64]
 [
 	0.004677
 	0.002963
 	-0.035612
 	0.026095
 	0.057679
 	…
 	-0.043745
 	0.042048
 	0.0
 	-0.040822
 	0.015209
 ],
 shape: (1_425,)
 Series: 'Close' [f64]
 [
 	0.0
 	0.0
 	0.0
 	0.0
 	0.0
 	…
 	0.0


In [21]:
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2005-04-22,520.664895,520.664895,423.591779,432.857849,3304177,0.0,0.0
1,2005-04-25,436.387792,439.917724,415.649444,419.002869,1021511,0.0,0.0
2,2005-04-26,422.091553,424.739001,410.795783,412.119507,680155,0.0,0.0
3,2005-04-27,412.119501,449.007275,407.530580,424.033020,1872361,0.0,0.0
4,2005-04-28,423.591803,435.505322,417.679156,421.650330,741207,0.0,0.0
...,...,...,...,...,...,...,...,...
4569,2023-10-26,36.000000,36.599998,34.650002,35.700001,770037,0.0,0.0
4570,2023-10-27,36.099998,36.950001,35.900002,36.599998,573842,0.0,0.0
4571,2023-10-30,36.799999,37.349998,36.250000,36.900002,400942,0.0,0.0
4572,2023-10-31,37.099998,37.450001,36.200001,36.400002,353270,0.0,0.0


In [22]:
temp = []
for i in range(len(stocklist)):
    temp.append(stocklist[i].hstack([logstock[i].alias("Log Return")]))
stocklist = temp

In [23]:
stocklist

[shape: (3_714, 9)
 ┌────────────┬────────────┬────────────┬───────────┬───┬──────────┬───────────┬────────┬───────────┐
 │ Date       ┆ Open       ┆ High       ┆ Low       ┆ … ┆ Volume   ┆ Dividends ┆ Stock  ┆ Log       │
 │ ---        ┆ ---        ┆ ---        ┆ ---       ┆   ┆ ---      ┆ ---       ┆ Splits ┆ Return    │
 │ str        ┆ f64        ┆ f64        ┆ f64       ┆   ┆ i64      ┆ f64       ┆ ---    ┆ ---       │
 │            ┆            ┆            ┆           ┆   ┆          ┆           ┆ f64    ┆ f64       │
 ╞════════════╪════════════╪════════════╪═══════════╪═══╪══════════╪═══════════╪════════╪═══════════╡
 │ 2008-10-06 ┆ 35.138197  ┆ 35.138197  ┆ 13.879588 ┆ … ┆ 23501600 ┆ 0.0       ┆ 0.0    ┆ -0.86899  │
 │ 2008-10-07 ┆ 14.055279  ┆ 16.690643  ┆ 12.232485 ┆ … ┆ 9113400  ┆ 0.0       ┆ 0.0    ┆ -0.062873 │
 │ 2008-10-08 ┆ 12.298368  ┆ 12.825442  ┆ 11.024609 ┆ … ┆ 2464384  ┆ 0.0       ┆ 0.0    ┆ -0.05506  │
 │ 2008-10-10 ┆ 10.936764  ┆ 10.936764  ┆ 9.509275  ┆ … ┆ 12079